# Create OMMX Artifact

In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
import uuid
from ommx.artifact import Artifact, ArtifactBuilder
from ommx.testing import SingleFeasibleLPGenerator, DataType

In [3]:
generator = SingleFeasibleLPGenerator(3, DataType.INT)
instance = generator.get_v1_instance()
state = generator.get_v1_state()

In [4]:
builder = ArtifactBuilder.new(f"ghcr.io/jij-inc/ommx/single_feasible_lp:{uuid.uuid4()}")
builder.add_instance(instance)
artifact = builder.build()

In [5]:
artifact.push()

INFO:ommx.artifact:Pushing: ghcr.io/jij-inc/ommx/single_feasible_lp:75edb44f-82c2-4869-9695-0b98e48c5979
INFO:ocipkg.distribution.client:POST https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/uploads/
INFO:ocipkg.distribution.client:PUT https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/upload/ca328d27-de37-4f3c-a13f-578a98632d70
INFO:ocipkg.distribution.client:POST https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/uploads/
INFO:ocipkg.distribution.client:PUT https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/upload/b5fddbe8-b5b5-4564-9534-d803c3be30a9
INFO:ocipkg.distribution.client:PUT https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/manifests/75edb44f-82c2-4869-9695-0b98e48c5979
